### TO DO - Aug 15th
* Merge tables by types of features: 
    * PDFEAT/DIAGFEAT
    * MHXGEN/GENPHYEX
    * PENNEURO/PENNEURO2
    * All the UPDRS
    * PASE/PASELT
    * All the non-motor assessments (add one column with category: olfaction, etc...)
    * _(Imaging & Medication data to do afterward). 
* DropNa? 
* calculate final ratio # of features/datapoints. 
* Wrote features_selection function.
* Add a field INFODT_date to convert INFODT into a datetime

In [1]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
def reading_csv_files(folder_path):
    # saving all the csv files in a dataframe
    import os
    import glob
    # glob.glob('*.csv') #find all the csv files in a pathname. 
    os.chdir(folder_path)
    csv_files = [i for i in glob.glob('*.csv')]

    # Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
    dict_files={}
    files_names=[]
    for files in csv_files:
        df=pd.read_csv(files,engine='python')
        basepath, filename=ntpath.split(files)
        files_names.append(filename)
        dict_files[filename]=df

    # Calculating the # of rows and features for each dataframe and storing them in a list of tuples
    shape_df=[]
    for i in range(0,len(dict_files.keys())):
        nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
        shape_df.append((nrows,ncols))

    return (shape_df,files_names,dict_files)

In [3]:
def number_of_patients(df):
    '''return the number of patients in each csv files'''
    if 'PATNO' in df.columns:
        patients=len(list(set(df['PATNO'])))
    else: 
        patients=0
    return patients

In [4]:
def number_of_events(df): 
    '''return the number of events covered by each csv file'''
    if 'EVENT_ID' in df.columns:
        events=len(list(set(df['EVENT_ID'])))
    else: 
        events=0
    return events

In [5]:
def features_selection(list_df_sel):
    '''do features selection for each df,sel in list_df_sel'''
    return df_sel

In [6]:
def patients_selection(df, threshold=6):
    '''return a dataframe containing only the patients with number of visits > threshold'''
    if 'PATNO' in df.columns: 
        visits_number_by_pat=df.groupby('PATNO').size().sort_values(ascending=False)
        mask_sel=visits_number_by_pat>=threshold
        patients_sel=list(mask_sel[mask_sel.values==True].index)
        df=df.loc[df['PATNO'].isin(patients_sel),:]
        df=df.sort_values('PATNO')
        return (patients_sel, df)
    else:
        return 0

In [41]:
def table_analysis(list_df,df_names,threshold=6): 
    '''return a dataframe with: 
    -as rows: list of df
    -as columns: 
        - total number of observations=rows
        - total number of features=columns after features_selection - features_selection function 
        - number of events
        - number of PD patients by using PRODROMA info - number_of_patients function
        - number of patients with # of visits > threshold - number_of_patients function
        - shape of final df after selections of rows and columns
        -number of NaN values in the final df
        NB: takes as input the df with features selection. 
    '''   
    # initialisation df
    col=['observations_tot','features','events_num','patients_num','patients_sel','len_df_sel','NaN_values']
    df_df=pd.DataFrame(columns=col,index=df_names)

    # remplissage of dataframe for each row
    for i,df in enumerate(list_df): 
        df_df.iloc[i,:]['observations_tot']=len(df)
        df_df.iloc[i,:]['features']=len(df.columns)
        df_df.iloc[i,:]['events_num']=number_of_events(df)
        df_df.iloc[i,:]['patients_num']=number_of_patients(df)
        
        if not patients_selection(df)==0:
            df_df.iloc[i,:]['patients_sel']=len(patients_selection(df,threshold=threshold)[0])
            df=patients_selection(df,threshold=threshold)[1]
        df_df.iloc[i,:]['len_df_sel']=len(df)
        df_df.iloc[i,:]['NaN_values']=(df.isnull().sum().sum())/(df_df.iloc[i,:]['len_df_sel']*df_df.iloc[i,:]['features'])
        
    return df_df

In [43]:
def INFODT_date(df):
    if 'INFODT' in df.columns: 
        df['INFODT_date']=df['INFODT'].apply(lambda x: datetime.strptime(x,'%m/%Y'))
    return df

## 1-Subject-Characteristics

In [8]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Patient_Status - actually useful for selecting only the PD patients in each table

PRODROMA=dict_files[files_names[0]]
print(number_of_patients(PRODROMA))
#print(list(set(PRODROMA['RECRUITMENT_CAT'])))
print(list(set(PRODROMA['ENROLL_CAT'])))
print(PRODROMA['ENROLL_CAT'].isnull().sum())
PD_cat=['PD','REGPD','GENPD']
mask=PRODROMA['ENROLL_CAT'].isin(PD_cat)
PD_patients=PRODROMA.loc[mask,'PATNO']
len(PD_patients)

### Screening__Demographics - useful
#### NOT TS DATA
SCREEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME',
       'CONSNTDT', 'APPRDX', 'CURRENT_APPRDX', 'P3GRP', 'BIRTHDT', 'GENDER',
       'HISPLAT', 'RAINDALS', 'RAASIAN', 'RABLACK', 'RAHAWOPI', 'RAWHITE',
       'RANOS', 'PRJENRDT']
SCREEN_sel=SCREEN[SEL]
print(number_of_patients(SCREEN))

2135
[nan, 'HC', 'PD', 'REGUN', 'REGPD', 'SWEDD', 'GENUN', 'PRODROMA', 'GENPD']
331
2144


## 5-Medical History
### PDFEAT is important - keep it.

In [18]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Diagnostic_Features.csv': useful: Feat-PD - 1670 patients.

DIAGFEAT=dict_files[files_names[1]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DFSTROKE', 'DFRSKFCT', 'DFPRESNT', 'DFRPROG', 'DFSTATIC', 'DFHEMPRK',
       'DFAGESX', 'DFOTHCRS', 'DFCRSCM', 'DFRTREMP', 'DFRTREMA', 'DFPATREM',
       'DFOTHTRM', 'DFTREMCM', 'DFRIGIDP', 'DFRIGIDA', 'DFAXRIG', 'DFUNIRIG',
       'DFTONE', 'DFOTHRIG', 'DFRIGCM', 'DFBRADYP', 'DFBRADYA', 'DFAKINES',
       'DFBRPLUS', 'DFOTHABR', 'DFABRCM', 'DFPGDIST', 'DFGAIT', 'DFFREEZ',
       'DFFALLS', 'DFOTHPG', 'DFPGCM', 'DFPSYCH', 'DFCOGNIT', 'DFDYSTON',
       'DFCHOREA', 'DFMYOCLO', 'DFOTHHYP', 'DFHYPCM', 'DFHEMTRO', 'DFPSHYPO',
       'DFSEXDYS', 'DFURDYS', 'DFBWLDYS', 'DFOCULO', 'DFEYELID', 'DFNEURAB',
       'DFDOPRSP', 'DFRAPSPE', 'DFBULBAR', 'DFCTSCAN', 'DFMRI', 'DFATYP']
DIAGFEAT_sel=DIAGFEAT[SEL]

### 'General_Medical_History.csv': useful: Feat-Gen - 2067 patients
#### NOT TS DATA...

MHXGEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MHCAT', 'MHROW', 'MHHX', 'MHACTRES', 'MHDIAGYR', 
     'PT_CODE', 'PT_NAME', 'HLT_NAME', 'SOCABBR1']
# I need probably to select less features here
MHXGEN_sel=MHXGEN[SEL]

### General_Physical_Exam: useful: Feat-Gen - 2047 patients
GENPHYEX=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PESEQ', 'PECAT', 'ABNORM']
GENPHYEX_sel=GENPHYEX[SEL]

### 'General_Neurological_Exam.csv': useful: Feat-Gen - 2048 patients
PENEURO=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MSRARSP', 'MSRACM', 'MSLARSP', 'MSLACM', 'MSRLRSP', 'MSRLCM',
       'MSLLRSP', 'MSLLCM', 'COFNRRSP', 'COFNRCM', 'COFNLRSP', 'COFNLCM',
       'COHSRRSP', 'COHSRCM', 'COHSLRSP', 'COHSLCM', 'SENRARSP', 'SENRACM',
       'SENLARSP', 'SENLACM', 'SENRLRSP', 'SENRLCM', 'SENLLRSP', 'SENLLCM',
       'RFLRARSP', 'RFLRACM', 'RFLLARSP', 'RFLLACM', 'RFLRLRSP', 'RFLRLCM',
       'RFLLLRSP', 'RFLLLCM', 'PLRRRSP', 'PLRRCM', 'PLRLRSP', 'PLRLCM']
PENEURO_sel=PENEURO[SEL]

### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD - 2049 patients
PENEURO2=dict_files[files_names[13]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CN1RSP','CN2RSP', 'CN346RSP','CN5RSP',
     'CN7RSP', 'CN8RSP', 'CN910RSP',
       'CN11RSP', 'CN12RSP']
PENEURO2_sel=PENEURO2[SEL]

### 'PD_Features.csv': useful: Feat-PD - caution: these are actually not timeseries but examination only done at screening (SC), BL & V04
### 1017 patients
PDFEAT=dict_files[files_names[14]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'SXMO',
       'SXYEAR', 'PDDXDT', 'DXTREMOR', 'DXRIGID', 'DXBRADY',
       'DXPOSINS', 'DXOTHSX','DOMSIDE']
PDFEAT_sel=PDFEAT[SEL]
#list(set(PDFEAT_sel['DXOTHCM'])) - see if we remove the comment feature or not.

### 'Vital_Signs.csv': useful: Feat-Gen
### 2047 patients
VITAL=dict_files[files_names[15]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'WGTKG', 'HTCM', 'TEMPC', 'SYSSUP', 'DIASUP', 'HRSUP',
       'SYSSTND', 'DIASTND', 'HRSTND']
VITAL_sel=VITAL[SEL]

In [34]:
# merging the 2 dataframes. 
PD_features=pd.merge(DIAGFEAT_sel,PDFEAT_sel,how='left',on=['PATNO','EVENT_ID','INFODT'])
len(PD_features)
# merge MHXGEN & GENPHYEX
Gen_Medical=pd.merge(MHXGEN_sel,GENPHYEX_sel,how='left',on=['PATNO','EVENT_ID','INFODT'])
len(Gen_Medical)
Gen_Medical=pd.merge(Gen_Medical,VITAL_sel,how='left',on=['PATNO','EVENT_ID','INFODT'])
# merge PENNEURO/PENNEURO2
Gen_Neuro=pd.merge(PENEURO_sel,PENEURO2_sel,how='inner',on=['PATNO','EVENT_ID','INFODT'])
len(Gen_Neuro)

6217

## 6-Motor-Assessments - all of them can be useful. 
#### But we might make the choice to not take all of them. 
#### For assessing the disease state (the labels), I think we should focus on the UPDRS1,3,&4 and take the other assessments as features

In [35]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv' - 2011 patients
NUPDR1SP=dict_files[files_names[0]]

### 'MDS_UPDRS_Part_I.csv' - 2010 patients
NUPDRS1=dict_files[files_names[2]]

### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv' - 2011 patients
NUPDRS2P=dict_files[files_names[11]]

### 'MDS_UPDRS_Part_III.csv' - 2010 patients
NUPDRS3=dict_files[files_names[7]]

### 'MDS_UPDRS_Part_IV.csv' - 860 patients
NUPDRS4=dict_files[files_names[5]]

### 'PASE_-_Household_Activity.csv' - 1248 patients
PASE=dict_files[files_names[3]]

### 'PASE_-_Leisure_Time_Activity.csv' - 1248 patients
PASELT=dict_files[files_names[10]]

### 'Modified_Schwab_+_England_ADL.csv' - 1837 patients
MODSEADL=dict_files[files_names[6]]

In [40]:
NUPDRS1_2P=pd.merge(NUPDRS1,NUPDRS2P,how='inner',on=['PATNO','EVENT_ID','INFODT'])
NUPDRS1_2P

,REC_ID_x,F_STATUS_x,PATNO,EVENT_ID,PAG_NAME_x,INFODT,NUPSOURC_x,NP1COG,NP1HALL,NP1DPRS,...,NP2HOBB,NP2TURN,NP2TRMR,NP2RISE,NP2WALK,NP2FREZ,ORIG_ENTRY_y,LAST_UPDATE_y,QUERY_y,SITE_APRV_y
0,231798601,V,3402,SC,NUPDRS1,07/2010,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,07/2010,2010-07-09 12:04:22.0,NaN,07/2010
1,233164301,V,3400,BL,NUPDRS1,07/2010,1.0,0.0,0.0,2.0,...,2.0,1.0,1.0,2.0,1.0,1.0,07/2010,2010-07-15 17:11:10.0,NaN,07/2010
2,230955201,V,3403,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,07/2010,2010-07-06 13:06:55.0,NaN,07/2010
3,234837501,V,3404,BL,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-23 11:36:57.0,NaN,07/2010
4,232712901,V,3406,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,2.0,0.0,3.0,0.0,0.0,0.0,07/2010,2010-07-14 08:56:34.0,NaN,07/2010
5,232592801,V,3400,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,07/2010,2010-07-13 16:48:22.0,NaN,07/2010
6,232528701,V,3401,BL,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-13 13:37:01.0,NaN,07/2010
7,231737401,V,3403,BL,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,2.0,1.0,0.0,07/2010,2010-07-09 09:28:01.0,NaN,07/2010
8,236118001,V,3405,BL,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-30 11:43:38.0,NaN,07/2010
9,237149401,V,3406,BL,NUPDRS1,08/2010,1.0,0.0,0.0,1.0,...,2.0,0.0,3.0,0.0,0.0,0.0,08/2010,2010-08-05 09:02:19.0,NaN,08/2010


## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

In [11]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'University_of_Pennsylvania_Smell_ID_Test.csv' - 1799 patients - OLFACTION
UPSIT=dict_files[files_names[0]]
UPSIT.head()

### 'Epworth_Sleepiness_Scale.csv - 1343 patients - SLEEP. 
EPWORTH=dict_files[files_names[2]]
EPWORTH.head()

### 'Letter_-_Number_Sequencing__PD_.csv' - 1341 patients. - MEMORY. 
LNSPD=dict_files[files_names[3]]
LNSPD.head()

### 'Olfactory_UPSIT.csv' - 4468 patients - OLFACTION. 
#### NB: only take the 2 last columns as features (TOTAL_CORRECT, UPSIIT_PRCNTGE)
OLFACT=dict_files[files_names[4]]
OLFACT.head()

### 'REM_Sleep_Disorder_Questionnaire.csv' - 1810 patients - SLEEP. 
REMSLEEP=dict_files[files_names[5]]
REMSLEEP.head()

### 'Cognitive_Assessments.csv' - 1172 patients
COGTIME=dict_files[files_names[6]]
COGTIME.head()

### 'SCOPA-AUT.csv' - 1345 patients - AUTOMATIC DYSFUNCTIONS
SCOPAAUT=dict_files[files_names[7]]
SCOPAAUT.head()

### Cognitive_Categorization.csv' - 1302 patients - DEMENTIA/MILD COGNITIVE IMPAIRMENT
COGCATG=dict_files[files_names[8]]
COGCATG.head()

### 'Hopkins_Verbal_Learning_Test.csv' - 1341 patients - MEMORY
HVLT=dict_files[files_names[9]]
HVLT.head()

### 'Montreal_Cognitive_Assessment__MoCA_.csv' - 2054 patients - MILD COGNITIVE IMPAIRMENT - PD 
MOCA=dict_files[files_names[10]]
MOCA.head()

### 'Semantic_Fluency.csv' - 1342 patients - Lexical/Semantic Fluency
SFT=dict_files[files_names[11]]
SFT.head()

### 'Geriatric_Depression_Scale__Short_.csv' - 1478 patients - DEPRESSION
GDSSHORT=dict_files[files_names[12]]
GDSSHORT.head()

### 'Symbol_Digit_Modalities.csv' - 1340 patients - COGNITIVE IMPAIRMENT. 
SDM=dict_files[files_names[13]]
SDM.head()

### 'State-Trait_Anxiety_Inventory.csv' - 1474 patients - DEPRESSION
STAI=dict_files[files_names[14]]
STAI.head()

### 'Benton_Judgment_of_Line_Orientation.csv' - 1341 patients - SPATIAL PERCEPTION/ORIENTATION 
BENTONOD=dict_files[files_names[15]]
BENTONOD.head()

### 'QUIP_Current_Short.csv' - 1346 patients - COMPULSIVE DISORDERS
QUIPCS=dict_files[files_names[16]]

In [29]:
list_df=[DIAGFEAT_sel,MHXGEN_sel,GENPHYEX_sel,PENEURO_sel,PENEURO2_sel,VITAL_sel,
NUPDRS1,NUPDRS2P,NUPDRS3,NUPDRS4,PASE,PASELT,MODSEADL,
UPSIT,EPWORTH,LNSPD,OLFACT,REMSLEEP,COGTIME,SCOPAAUT,COGCATG,HVLT,
MOCA,SFT,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS]

df_names=['DIAGFEAT','MHXGEN','GENPHYEX','PENEURO','PENEURO2','VITAL',
'NUPDRS1','NUPDRS2P','NUPDRS3','NUPDRS4','PASE','PASELT','MODSEADL',
'UPSIT','EPWORTH','LNSPD','OLFACT','REMSLEEP','COGTIME','SCOPAAUT','COGCATG','HVLT',
'MOCA','SFT','GDSSHORT','SDM','STAI','BENTONOD','QUIPCS']

dataset_analysis_maintables=table_analysis(list_df,df_names)
dataset_analysis_maintables

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars


,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
DIAGFEAT,5739,59,19,1670,320,2349,0.100779
MHXGEN,63933,14,2,2067,2067,63933,0.383794
GENPHYEX,24382,8,8,2047,2047,24382,0.000148675
PENEURO,6217,41,15,2048,440,2877,0.428563
PENEURO2,6218,14,15,2049,440,2877,0
VITAL,12762,14,21,2047,910,10544,0.0818407
NUPDRS1,11236,17,20,2010,856,8859,0.0588567
NUPDRS2P,11235,24,20,2011,857,8864,0.0418829
NUPDRS3,12696,51,20,2010,896,10440,0.0989219
NUPDRS4,4541,16,19,860,405,3464,0.0626443


## 2-Biospecimen

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/2-Biospecimen/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Genetic_Results_Tests - useful: Feat-bio - 1116 patients
#### NOT TS DATA

MUTRSLT=dict_files[files_names[3]]
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'GENECAT', 'LRRKCD', 'MUTRSLT']
MUTRSLT_sel=MUTRSLT[SEL]
print(number_of_patients(MUTRSLT))
MUTRSLT_sel.head()

### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
#### Understand if the cohort of patient IDs are different from the classic PPMI cohort.
#### NOT TS DATA.

### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
Biospecimen_Analysis=dict_files[files_names[6]]

SEL=['PATNO', 'DIAGNOSIS', 'CLINICAL_EVENT', 'TYPE', 'TESTNAME',
       'TESTVALUE','RUNDATE', 'PROJECTID']
print(number_of_patients(Biospecimen_Analysis))
Biospecimen_Analysis_sel=Biospecimen_Analysis[SEL]

### Lumbar_Puncture_Sample_Collection - useful: Feat-bio - 1344 patients
LUMBAR=dict_files[files_names[9]]

### Only features useful I think: white cell blood count, red blood cell count, total glucose, total protein.

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'FASTSTAT','PDMEDYN', 'PDMEDDT', 'PDMEDTM',
     'SMPDSCRD','RBCRSLT','WBCRSLT','TOPRRSLT','TGLCRSLT','SPFI','FLUORO']
print(number_of_patients(LUMBAR))
#print(len(patients_selection(LUMBAR,6)[0]))
LUMBAR_sel=LUMBAR[SEL]

## 3-Enrollment - useless?

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Primary_Diagnosis.csv' - useful: Feat-Gen - 789 patients... (only PD patients?)
PRIMDXPD=dict_files[files_names[8]]

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PRIMDIAG', 'OTHNEURO']
print(number_of_patients(PRIMDXPD))
PRIMDXPD_sel=PRIMDXPD[SEL]

files_names

# Datasets to investigate

### Blood_Chemistry__Hematology - Contains information about patient blood. 
### Ask doctors if really useful. 

In [ ]:
COVANCE=dict_files[files_names[12]]
#COVANCE.head()

### IUSM_Catalog - DNK - most likely useless
#### seems ti be only metadata on the lab collections.
Look @ the PPMI doc to understand what is this table

In [ ]:
IUSMCAT=dict_files[files_names[14]]
#IUSMCAT.head()

# Additionnal data that will be used later on (imaging data and medication data)

## 4-Imaging - dataset not to be used for the time being.
### Info about Imaging Data pages 98 to 101 in the study protocol. 

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/4-Imaging/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### FBB_Analysis_Data.csv
FBBANAL=dict_files[files_names[0]]
# no selection to do. 

### AV-133_Imaging.csv
AVIMAG=dict_files[files_names[2]]
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'PETCMPLT','VMATRSLT']
AVIMAG_sel=AVIMAG[SEL]

### DaTscan_Imaging.csv
DATSCAN=dict_files[files_names[3]]
DATSCAN.columns
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DATSCAN','VSINTRPT','VSRPTELG']
DATSCAN_sel=DATSCAN[SEL]

### AV-133_SBR_Results.csv
ind_av133_sbr=dict_files[files_names[5]]
col_old=list(ind_av133_sbr.columns)
col_new=['PATNO',
 'EVENT_ID',
 'INFODT',
 'TIMEPOINT',
 'RCAUD-S',
 'RPUTANT-S',
 'RPUTPOST-S',
 'LCAUD-S',
 'LPUTANT-S',
 'LPUTPOST-S',
 'update_stamp']
col=dict(zip(col_old,col_new))
ind_av133_sbr.rename(columns=col,inplace=True)
SEL=['PATNO', 'EVENT_ID', 'INFODT', 'RCAUD-S', 'RPUTANT-S',
       'RPUTPOST-S', 'LCAUD-S', 'LPUTANT-S', 'LPUTPOST-S']
ind_av133_sbr_sel=ind_av133_sbr[SEL]

### Magnetic_Resonance_Imaging.csv
MRI=dict_files[files_names[6]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MRICMPLT', 'MRIWDTI', 'MRIWRSS', 'MRIRSLT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG']
MRI_sel=MRI[SEL]

### DTI_Regions_of_Interest
DTIROI=dict_files[files_names[7]]
SEL=['PATNO', 'PAG_NAME', 'INFODT', 'Measure','ROI1', 'ROI2',
       'ROI3', 'ROI4', 'ROI5', 'ROI6', 'REF1', 'REF2']
DTIROI_sel=DTIROI[SEL]

### DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv - not found in data dictionary...
SPECT_vis_assess=dict_files[files_names[8]]
#SPECT_vis_assess.head()

In [ ]:
list_df=[FBBANAL,AVIMAG_sel,DATSCAN_sel,ind_av133_sbr_sel,MRI_sel,DTIROI_sel]
df_names=['FBBANAL','AVIMAG','DATSCAN','ind_av133_sbr','MRI','DTIROI']
dataset_analysis_imaging=table_analysis(list_df,df_names,threshold=3) #threshold=3 to have more 
dataset_analysis_imaging

## Medications data - to be used later on.

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

In [ ]:
### Use of PD Medication: useful: Meds - but not to be used for now.
PDMEDUSE=dict_files[files_names[4]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG', 'ONOTHER', 'FULNUPDR', 'PDMEDDT',
       'PDMEDTM', 'NUPDRTM']
PDMEDUSE_sel=PDMEDUSE[SEL]

### 'Concomitant_Medications.csv': useful: Meds - no time to exploit this stuff. 
CMED=dict_files[files_names[6]]

### 'Current_Medical_Conditions_Log.csv': Meds
CURRCOND=dict_files[files_names[8]]
CURRCOND.columns
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'SEQNO',
       'CONDCAT', 'DIAGYR','CONDTERM', 'RESOLVD', 'RESYR',
       'HLGTNAME', 'HLT_NAME','PT_CODE','SOCABBR1', 'VMEDDRA']
CURRCOND_sel=CURRCOND[SEL]

### 'Adverse_Event_Log.csv': useful: Meds - but not to be used for now.
#### Measures the adverse events occurring after LUMBAR PUNCTURE, SPECT, AV-133 PET Imaging, 
#florbetaben imaging, skin biopsy.  -USELESS

AE=dict_files[files_names[9]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'AESEQ',
       'AETERM', 'STARTDT','STOPDT', 'AESEVER', 'SAE',
       'AERELAT', 'RELDSCAN', 'RELLP', 'RELPRCDR', 'RELAV133', 'RELFB',
       'RELSKBIO', 'AEOUTCOM', 'AEFLUP',
       'PT_CODE', 'PT_NAME', 'HLT_NAME', 'HLGTNAME', 'SOCABBR1', 'VMEDDRA']
AE_sel=AE[SEL]

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds - but not to be used for now.
SURGPD=dict_files[files_names[12]]
SURGPD.columns
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDSURG', 'PDSURGDT', 'PDSURGTP']
SURGPD_sel=SURGPD[SEL]

In [ ]:
list_df=[PDMEDUSE_sel,CURRCOND_sel,AE_sel,SURGPD_sel]
df_names=['PDMEDUSE','CURRCOND','AE','SURGPD']
dataset_analysis_meds=table_analysis(list_df,df_names)
dataset_analysis_meds

### Family_History_PD - most likely useful but not to be used for now.
#### Can be useful for detecting early PD

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

FAMHXPD=dict_files[files_names[2]]
#FAMHXPD.head()

### 'Prodromal_Diagnostic_Questionnaire.csv': useful: Feat-Gen (PRIMDIAG & PSLVL2) 
#### NB: PSLV2 evaluates the proba of the patient being PD: might be useful for early PD. But not to be used in the main analysis as a feature (only for Prodromal & Genetic Cohort - i.e 800 subjects) 

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

PRODIAG=dict_files[files_names[0]]
SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'PRIMDIAG',
       'OTHNEURO', 'PSLVL2']
PRODIAG_sel=PRODIAG[SEL]
PRODIAG_sel.head()

### 'Signature_Form.csv' - most likely useful: Meds - but not to be used for now.
#### Can be useful for the meds log

In [ ]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

SIG=dict_files[files_names[10]]
SIG.head()

## Datasets with two few data points (not enough patients)

### IPSC_Labs: useful - Feat-bio: blood, liver disease, heparin(?), myeloma(?) stuff (linked to Med dose history)
### ONLY 137 patients...

In [ ]:
### ISPC_Labs: useful - Feat-bio
#### Select only history of liver disease and multiple myeloma (can be useful for treatments strategies)
IPSCLAB=dict_files[files_names[5]]
#IPSCLAB.head()
IPSCLAB.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'PDMEDDT', 'PDMEDTM', 'WARFBEF', 'HEPBEF', 'HXLIVER',
       'HXMMYLMA', 'BLDLTH', 'LTHTM', 'LTHINVN', 'BLDSST', 'BLDCPT']
# we might be able to remove some features. 
print(number_of_patients(IPSCLAB))
IPSCLAB_sel=IPSCLAB[SEL]
IPSCLAB_sel.head()
#list(set(IPSCLAB_sel['EVENT_ID']))

### 'Clinical_Diagnosis_and_Management.csv': Feat-PD - only 73 patients... (just for SWEDD). 

In [ ]:
### 'Clinical_Diagnosis_and_Management.csv': Feat-PD
CLINDX=dict_files[files_names[2]]
CLINDX.head()
#CLINDX.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PSLVL', 'DCNOMTR', 'DCRTREM', 'DCRIGID', 'DCBRADY', 'DFPGDIST',
       'DFOTH', 'PRIMDIAG', 'OTHNEURO', 'CLDXCHNG', 'DXFDTIMG',
       'DXFCLSGN', 'DXFRSPMD', 'DXFNATHX', 'DXFOTH', 'DXFOTHCM', 'MNGCHNG',
       'MNGTRTSX', 'MNGTRIAL', 'MNGTRTOT', 'MNGDXTST', 'MNGNOTRT',
       'NEURWKDX']
print(number_of_patients(CLINDX))
CLINDX_sel=CLINDX[SEL]
CLINDX_sel.head()
#list(set(CLINDX_sel['EVENT_ID']))

### 'TAP-PD_OPDM_Assessment.csv' - only 173 patients...


In [ ]:
TAPOPDM=dict_files[files_names[8]]
print(number_of_patients(TAPOPDM))
TAPOPDM.head()